# Mapping unit from old files

In [1]:
import numpy as np
import pandas as pd

In [64]:


# Load the datasets
transaction=pd.read_excel("transaction_mapped.xlsx") #fully mapped new transaction files with dummies no portion
past_transaction = pd.read_excel("R_H_sorted_and_merged.xlsx", sheet_name='R')#fully mapped old transaction files with dummies and portion

In [65]:
past_transaction.columns

Index(['PID', 'FULL_DATE', 'TRANSACTION_HEADER_KEY', 'STORE_NUMBER', 'AREA',
       'DEPARTMENT_NAME', 'CATEGORY_NAME', 'SECTION_NAME', 'SUB_SECTION_NAME',
       'PRIMARY_UPC', 'PRODUCT_DESCRIPTION', 'SIZE', 'UNIT_OF_MEASURE',
       'SALES', 'UNITS_WEIGHT', 'FNDDS_SCHNUCKS_mapped', 'Weight',
       'WEIGHT_final', 'Unnamed: 18', 'UNIT_OF_MEASURE_mapped',
       'UNITS_PACKAGES', 'AD_TYPE_DESCRIPTION', 'AVG_PRICE_PER_MEASURE',
       'AVG_PRICE_PER_PACK', 'HUMAN_CONSUMABLE_CODE', 'DIETITIAN_PICK',
       'WIC_FLAG', 'FOOD_STAMP_FLAG', 'FV_DUMMY1', 'FV_DUMMY2'],
      dtype='object')

In [66]:
transaction.columns

Index(['PID', 'FULL_DATE', 'TRANSACTION_HEADER_KEY', 'STORE_NUMBER', 'AREA',
       'DEPARTMENT_NAME', 'CATEGORY_NAME', 'SECTION_NAME', 'SUB_SECTION_NAME',
       'PRIMARY_UPC', 'PRODUCT_DESCRIPTION', 'SIZE', 'UNIT_OF_MEASURE',
       'SALES', 'UNITS_WEIGHT', 'UNITS_PACKAGES', 'AD_TYPE_DESCRIPTION',
       'AVG_PRICE_PER_MEASURE', 'AVG_PRICE_PER_PACK', 'HUMAN_CONSUMABLE_CODE',
       'DIETITIAN_PICK', 'WIC_FLAG', 'FOOD_STAMP_FLAG', 'FV_DUMMY1',
       'FV_DUMMY2'],
      dtype='object')

In [67]:
FV=past_transaction[past_transaction['FV_DUMMY1'] == 1]
FV1=transaction[transaction['FV_DUMMY1'] == 1]

In [68]:
FV['UNIT_OF_MEASURE'].value_counts()

UNIT_OF_MEASURE
LB    3470
OZ    3128
EA    1977
1N     172
Name: count, dtype: int64

In [69]:
target = FV[~FV['UNIT_OF_MEASURE'].isin(['LB', 'OZ'])]


In [56]:
target['UNITS_WEIGHT']=target['UNITS_WEIGHT']/target['UNITS_PACKAGES']/target['SIZE']

/var/folders/rc/f8gf4g817rq5cl8_jh8vtny00000gn/T/ipykernel_66567/923123955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['UNITS_WEIGHT']=target['UNITS_WEIGHT']/target['UNITS_PACKAGES']/target['SIZE']


In [71]:
unit_old=target[['PRODUCT_DESCRIPTION','UNIT_OF_MEASURE','UNITS_WEIGHT']].drop_duplicates()

In [72]:
FV_measure=FV1[['PRODUCT_DESCRIPTION','UNIT_OF_MEASURE']].drop_duplicates()

In [59]:
FV_measure

,PRODUCT_DESCRIPTION,UNIT_OF_MEASURE
2,Sweet Potatoes Product Of U.S.A.,LB
3,Green Cabbage Product Of U.S.A,LB
9,Fresh Bananas,LB
18,Mustard Greens Product Of U.S.A.,EA
19,Nature Greens - Curly Mustard Greens Bag (16 Oz),OZ
...,...,...
53266,Schnucks - Pieces And Stems Mushrooms No Salt ...,OZ
53470,Bella Sun Luci - Sun Dried Julienne Cut Tomato...,OZ
53755,Pictsweet - Speckled Butter Beans (12 Oz),OZ
54059,Thats Smart - Sweet Peas (15 Oz),OZ


In [73]:
FV_list1= FV_measure.merge(unit_old, on='PRODUCT_DESCRIPTION', how='left')

In [61]:
# Ensure numeric, but keep NaN if conversion fails
FV_list1['UNITS_WEIGHT'] = pd.to_numeric(FV_list1['UNITS_WEIGHT'], errors='coerce')

# Keep NaN, drop only inf, -inf, and negatives
FV_list1 = FV_list1[
    (FV_list1['UNITS_WEIGHT'] != np.inf) & 
    (FV_list1['UNITS_WEIGHT'] != -np.inf) & 
    ((FV_list1['UNITS_WEIGHT'].isna()) | (FV_list1['UNITS_WEIGHT'] >= 0))
].reset_index(drop=True)



In [75]:
FV_list1

,PRODUCT_DESCRIPTION,UNIT_OF_MEASURE_x,UNIT_OF_MEASURE_y,FNDDS_SCHNUCKS_mapped
0,Sweet Potatoes Product Of U.S.A.,LB,NaN,NaN
1,Green Cabbage Product Of U.S.A,LB,NaN,NaN
2,Fresh Bananas,LB,NaN,NaN
3,Mustard Greens Product Of U.S.A.,EA,EA,32.0
4,Nature Greens - Curly Mustard Greens Bag (16 Oz),OZ,NaN,NaN
...,...,...,...,...
1037,Schnucks - Pieces And Stems Mushrooms No Salt ...,OZ,NaN,NaN
1038,Bella Sun Luci - Sun Dried Julienne Cut Tomato...,OZ,NaN,NaN
1039,Pictsweet - Speckled Butter Beans (12 Oz),OZ,NaN,NaN
1040,Thats Smart - Sweet Peas (15 Oz),OZ,NaN,NaN


In [74]:
FV_list1.to_excel("FV_unit_old.xlsx", index=False) #list of FV products with their corresponding unit mapped from old files

In [17]:
duplicates = FV_list1.groupby('PRODUCT_DESCRIPTION').size()
duplicates = duplicates[duplicates > 1]
duplicates


PRODUCT_DESCRIPTION
Carrots 2 Lb Bag                                                2
Cotton Candy Grapes                                             2
Dole - Fruit Bowls Diced Pears In 100% Fruit Juice (4 Count)    2
Green Bell Pepper Product Of U.S.A. & Canada                    2
Green Kiwi                                                      2
Honeydew Melon                                                  2
Limes                                                           4
Microwaveable Sweet Potatoes                                    2
Organic Celery Hearts                                           2
Organic Yellow Onions 3 Lb Bag                                  2
Schnucks - Freshly Frozen Mini Corn On The Cob (12 Count)       2
Sugar Kiss Melons                                               2
dtype: int64

In [18]:
# Find duplicated PRODUCT_DESCRIPTION after merge
dupes = FV_list1[FV_list1.duplicated(subset=['PRODUCT_DESCRIPTION'], keep=False)]

# Show them
dupes=dupes.sort_values('PRODUCT_DESCRIPTION')
dupes.to_excel("FV_unit_dupe.xlsx", index=False)

# Calculate portion by FV unit mappped

In [34]:
import pandas as pd
transaction=pd.read_excel("transaction_mapped.xlsx") #fully mapped new transaction files with dummies no portion
FV_list=pd.read_excel("FV_unit.xlsx") #fully unit mapped fv product list

In [35]:
note=transaction[['PRODUCT_DESCRIPTION','FV_DUMMY1','FV_DUMMY2']]

In [36]:
FV_list = FV_list.drop_duplicates(subset=['PRODUCT_DESCRIPTION'], keep='first')

In [37]:
FV_list.columns

Index(['PRODUCT_DESCRIPTION', 'UNIT_OF_MEASURE', 'UNIT_OF_MEASURE_mapped',
       'FNDDS_SCHNUCKS'],
      dtype='object')

In [38]:
FV_list = FV_list[['PRODUCT_DESCRIPTION', 'UNIT_OF_MEASURE_mapped', 'FNDDS_SCHNUCKS']]

In [39]:
transaction=transaction.merge(FV_list, on='PRODUCT_DESCRIPTION', how='left')

set standard portion according to dummy 2

In [40]:
# Create 'standard_portion' with default value ( NaN)
transaction['standard_portion'] = None  

# Assign based on conditions
transaction.loc[transaction['FV_DUMMY2'].isin([1, 2, 3, 6, 7, 8]), 'standard_portion'] = 120
transaction.loc[transaction['FV_DUMMY2'].isin([4, 5]), 'standard_portion'] = 200
transaction.loc[transaction['FV_DUMMY2'] == 9, 'standard_portion'] = 30


set transfer_g to transfer all types of 'UNIT_OF_MEASURE' into 'g'

In [41]:
# Create 'transfer_g' with default value ( NaN)
transaction['transfer_g'] = None  
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'OZ'), 'transfer_g'] = 28.35
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'LB'), 'transfer_g'] = 453.59
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE_mapped'] == 'lb'), 'transfer_g'] = 453.59
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE_mapped'] == 'oz'), 'transfer_g'] = 28.35
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE_mapped'] == 'g'), 'transfer_g'] = 1

set UNITS_WEIGHT of products of EA and 1N = 'FNDDS_SCHNUCKS' mapped

In [42]:
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'].isin(['EA', '1N'])), 'UNITS_WEIGHT'] = transaction['FNDDS_SCHNUCKS']


Calculate: Portion= SIZE * UNITS_WEIGHT* UNIT_PACKAGE * transfer_g/standard portion

In [43]:
transaction['portion'] = transaction['SIZE']*transaction['UNITS_WEIGHT']*transaction['UNITS_PACKAGES']*transaction['transfer_g']/transaction['standard_portion']

create column final_weight for comparison

In [44]:
transaction['final_weight'] = None  
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'OZ'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_WEIGHT']
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'LB'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_WEIGHT']
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == '1N'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_PACKAGES']*transaction['FNDDS_SCHNUCKS']
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'EA'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_PACKAGES']*transaction['FNDDS_SCHNUCKS']


In [46]:
transaction.to_excel("transaction_portion.xlsx", index=False)